# 淘宝APP用户行为数据分析

![png](./pics/mind.png)

## 一.背景分析

淘宝网是中国深受欢迎的网购零售平台，是亚太地区较大的网络零售、商圈，由阿里巴巴集团在2003年5月创立。

这份数据下载自天池数据集：
https://tianchi.aliyun.com/dataset/dataDetail?dataId=46

数据为淘宝APP2014.11.18～2014.12.18的数据。2014年是阿里巴巴集团移动商务快速发展的一年。

数据主要包含用户在淘宝APP上包括点击、收藏、加购、购买的行为数据，其中各变量描述如下：

![png](./pics/v.png)



## 二.目标确定

1. 对运营情况进行监控，找出变动（上升/下降）的原因，防止异常事件发生，同时找出优化运营或产品的方法。

比如找出什么时间上新，什么时间提醒用户进行购买。

2. 精细化运营

对用户进行分类，针对不同用户采取不同营销措施。通过计算用户购买的频次，来对用户购买的活跃程度进行分类。

3. 发现用户喜欢的商品
通过对数加权的方法计算商品热度，了解用户喜好。

## 三. 数据加载及预处理

In [1]:
import pandas as pd
import numpy as np

In [2]:
users = pd.read_csv('tianchi_mobile_recommend_train_user.csv')
users.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21


In [3]:
users.shape

(12256906, 6)

### 1. 重复值检测及处理

In [10]:
users.duplicated().sum()

4092866

In [19]:
users.drop_duplicates(inplace=True)

In [15]:
display(users.duplicated().sum())
display(users.shape)

0

(8164040, 6)

### 2. 空缺值检测

In [18]:
def null_info(users):
    info = pd.DataFrame(users.isnull().sum()).T.rename(index={0:'null values (nb)'})
    info=info.append(pd.DataFrame(users.isnull().sum()/users.shape[0]*100).T.rename(index={0:'null values (%)'}))
    display(info)
null_info(users)

,user_id,item_id,behavior_type,user_geohash,item_category,time
null values (nb),0.0,0.0,0.0,4.308015e+06,0.0,0.0
null values (%),0.0,0.0,0.0,5.276818e+01,0.0,0.0


### 3. 类型检测及转化

In [27]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8164040 entries, 0 to 12256904
Data columns (total 6 columns):
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             object
dtypes: int64(4), object(2)
memory usage: 436.0+ MB


In [20]:
users.time = pd.to_datetime(users.time)

In [21]:
users['hour'] = users.time.dt.hour
users.head(1)

,user_id,item_id,behavior_type,user_geohash,item_category,time,hour
0,98047837,232431562,1,NaN,4245,2014-12-06 02:00:00,2


In [24]:
users['week'] = users.time.dt.week
users.head(1)

,user_id,item_id,behavior_type,user_geohash,item_category,time,hour,week
0,98047837,232431562,1,NaN,4245,2014-12-06 02:00:00,2,49


In [25]:
users['date'] = users.time.dt.date
users.head(1)

,user_id,item_id,behavior_type,user_geohash,item_category,time,hour,week,date
0,98047837,232431562,1,NaN,4245,2014-12-06 02:00:00,2,49,2014-12-06


In [26]:
users.item_category = users.item_category.astype('category')

In [27]:
# 保存处理后的数据
users_c = users.to_csv('users_c.csv',index=False)

## 四. 数据分析
### 1. 流量分析
- PV(Page View)：页面访问量
- UV(Unique Visitor)：独立访客数

![png](./pics/PVUV_day.png)

** 分析 **

通过监控每天PV和UV，可以看到平时它们的变化量都不大，但到了12月11日时出现了一个上升趋势，且12月12日达到峰值，说明“双十二”购物节的促销活动确实起到了吸引用户的作用。同时PV的增长了近50%，而相比PV，UV仅增长了约10%，可见“双十二”主要带来了老用户的大量回访。在服务器上也需要进行加强。

![png](./pics/PVUV_hour.png)

** 分析 **

通过监控每个小时PV和UV，可以发现每天的0～7点用户购买较低，其中凌晨4点达到最低值。中午和下午的独立用户访问量并不会有太大区别，18点之后PV和UV均有上升趋势，到21点左右达到峰值，因此如果在这个时候上新或者进行新活动，是最容易被用户关注到。

### 2. 活跃度分析

相比流量，活跃度更能衡量用户粘性，

这里的活跃用户定义为：进行商品收藏，商品加入购物车或者购买商品的用户。

- DAU(Daily Active User)：日活跃用户数量。
- HAU：每小时活跃用户数

![png](./pics/DAU.png)

** 分析 **

这里DAU与UV类似，但DAU更能反映用户对各类营销活动所采取的行动。12月12日达到峰值，但DAU比UV的增长幅度要大，说明“双十二”不仅仅带来了大量访问，而且大多数营销措施都取得了一定效果。


#### 对比平时和双十二每小时用户的活跃情况
![png](./pics/HAU_normal.png)

![png](./pics/HAU_12.png)

** 分析 **

对比平时和双十二每小时用户的活跃情况可以发现，平时在晚上21点左右的时候用户最活跃，但双十二这天出现了两个峰值，0点和晚上21～22点。可见，大促和限时抢购对用户确实起到了非常大的吸引力。同时，双十二这天在任何一个时间段的用户活跃数都比平时要高。

### 3. 转换率分析

#### 收藏率/加购率/购买率
使用漏斗模型进行分析

主要分析收藏，加购（即加入购物车），购买的日转化率以及小时转化率，并找出转换率最高/最低的时间点，以及导致的原因。

用户行为流程拆解：
1. 点击 -> 加入收藏夹
2. 点击（包括从普通页面点击以及从收藏夹点击） -> 加入购物车 -> 购买

计算指标确定：
1. 收藏率 = 收藏量 / 点击量
2. 加购率 = 加入购物车量 / 点击量
3. 购买率 = 购买量 / 加入购物车量

可以使用python计算，但为了使用Tableau画出更好看的图，这里使用Tableau直接计算，计算公式如下：

- 点击量：
SUM(IF [behavior_type]==1 THEN 1 
ELSEIF [behavior_type]!=1 THEN 0 END)
- 收藏量：
SUM(IF [behavior_type]==2 THEN 1 
ELSEIF [behavior_type]!=2 THEN 0 END)
- 加入购物车量:
SUM(IF [behavior_type]==3 THEN 1 
ELSEIF [behavior_type]!=3 THEN 0 END)
- 购买量:
SUM(IF [behavior_type]==4 THEN 1 
ELSEIF [behavior_type]!=4 THEN 0 END)

收藏率,加购率,购买率使用以上计算结果相除。

![png](./pics/rate_day.png)

** 分析 ** 

”双十二“这天，加购率和购买率都有所提升，但收藏率却有所降低，而在平时平均收藏率要低于平均加购率，说明在这天用户购物冲动比平时要高，平时犹豫买或不买的物品，在这天购买的可能性要大于平时。12月11日的购买率最低，说明大部分用户都把商品囤起来，等到12号在支付。而到了12月13日，又恢复到平时的情况。

![png](./pics/rate_hour.png)

** 分析 ** 

虽然用户活跃度通常都是晚上21点左右才达到峰值，但从转换率来说从21点持续到凌晨5点，收藏率和加购率都出于高峰时段，说明虽然活跃用户少但活跃的用户中最有可能把商品进行收藏或者加入购物车；从购买率来看，晚上看中的商品用户并不会立刻进行购买，而是到了早上11点左右进行购买，因此对于收藏或加购的商品，我们可以在上午11点左右通过平台、短信、邮件进行提醒购买。



### 4. 复购率
复购率的定义是：在某时间窗口内消费两次及以上的用户在总消费用户的占比。

In [3]:
# 加载清洗后的数据
users_c = pd.read_csv('users_c.csv')

In [5]:
# 发生购买行为的用户
df_pay = users_c[users_c.behavior_type==4]

In [92]:
# 计算每个用户购买的天数
df_pay_1 = df_pay.groupby(['user_id','date']).agg({'time': lambda x:x.count()})
df_pay_2 = df_pay_1.groupby('user_id').agg({'time':lambda x:x.count()}).rename(columns={'time':'pay_num'})
df_pay_2.head()

,pay_num
user_id,
4913,5
6118,1
7528,6
7591,9
12645,4


In [ ]:
df_pay_2.to_csv('pay_num.csv')

![png](./pics/pay_num.png)

** 分析 **

上图表示在发生过购买行为的用户中，一个月内购买天数的分布。

购买天数为1的用户占比最高，为12.83%。如果以月份作为时间窗口，假设每个用户每天购买次数为1（每笔订单可包含多个商品），那么复购率为1-12.83%=87.17%。同时还可以与其他月份的复购率做对比。

### 5. 用户生命周期
用户生命周期：第一次消费到最后一次消费之间的时间间隔。

由于只有一个月的数据，因此把这个月内第一次消费时间当作第一次消费时间，这个月内的最后一次消费当作最后一次消费时间。

In [100]:
# 计算生命周期
df_pay['date'] = pd.to_datetime(df_pay.date)

date_min = df_pay.groupby('user_id').date.min()
date_max = df_pay.groupby('user_id').date.max()
life_time = date_max - date_min
life_time.head()

/Users/jia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


user_id
4913    15 days
6118     0 days
7528    21 days
7591    18 days
12645   17 days
Name: date, dtype: timedelta64[ns]

In [112]:
df_life_time = pd.DataFrame(life_time)
df_life_time.rename(columns={'date':'life_time'},inplace=True)
df_life_time.to_csv('life_time.csv')

![png](./pics/life_time.png)

** 分析 **

上图表示在发生过购买行为的用户中,首次/末次购买时间的间隔时间。

排除只有一天有购买行为的用户（首末次购买时间间隔为0天），间隔为24、27、28、29天的用户数最多。

由于只有一个月的数据，因此这里求出的生命周期有一定局限性，如果有更长周期的数据，那么通过求出用户生命周期，我们可以知道间隔用户第一次购买的第几天采取营销措施，能延长用户的生命周期。

### 6. 用户分类
通过计算用户购买的频次，来对用户购买的活跃程度进行分类。

用数字1，2来代表用户的活跃情况：
- 1: 普通活跃用户
- 2: 非常活跃用户

In [44]:
active = pd.read_csv('pay_num.csv')
active.describe()

,user_id,pay_num
count,8.886000e+03,8886.000000
mean,7.152087e+07,5.536912
std,4.120719e+07,4.021963
min,4.913000e+03,1.000000
25%,3.567731e+07,2.000000
50%,7.238800e+07,5.000000
75%,1.071945e+08,8.000000
max,1.424559e+08,30.000000


In [49]:
# 1: 普通活跃用户
# 2：非常活跃用户
active['group'] = active.pay_num.apply(lambda x:2 if x>5 else 1)
active.head(2)

,user_id,pay_num,group
0,4913,5,1
1,6118,1,1


### 7. 商品热度
通过统计用户行为数据,计算不同类别商品的热度。

使用对数加权的方法进行热度计算，click, collect, add-to-cart, payment的权重依次增大。

In [68]:
item_heat = users_c[['item_category','behavior_type']]
# 计算每个商品被点击、收藏、加入购物车和付款的次数
cl=item_heat[item_heat.behavior_type==1].groupby(by='item_category',as_index=False).count().rename(columns={'behavior_type':'click'})
co=item_heat[item_heat.behavior_type==2].groupby(by='item_category',as_index=False).count().rename(columns={'behavior_type':'collect'})
atc=item_heat[item_heat.behavior_type==3].groupby(by='item_category',as_index=False).count().rename(columns={'behavior_type':'add-to-cart'})
pay=item_heat[item_heat.behavior_type==4].groupby(by='item_category',as_index=False).count().rename(columns={'behavior_type':'payment'})
# 合并计算结果，构建DataFrame
item_heat = pd.merge(cl,co,on='item_category',how='outer')
item_heat = pd.merge(item_heat,atc,on='item_category',how='outer')
item_heat = pd.merge(item_heat,pay,on='item_category',how='outer')
item_heat.fillna(0,inplace=True)
item_heat.head()

,item_category,click,collect,add-to-cart,payment
0,2,1.0,0.0,0.0,0.0
1,3,2.0,0.0,0.0,0.0
2,4,1.0,0.0,0.0,0.0
3,6,9.0,0.0,0.0,0.0
4,8,833.0,9.0,4.0,4.0


In [69]:
# 使用对数加权的方法进行热度计算，click, collect, add-to-cart, payment的权重依次增大
item_heat['click'] = np.log(item_heat['click']+0.0001)
item_heat['collect'] = np.log(item_heat['collect']+0.0001)*2
item_heat['add-to-cart'] = np.log(item_heat['add-to-cart']+0.0001)*3
item_heat['payment'] = np.log(item_heat['payment']+0.0001)*4
item_heat['heat'] = item_heat['click']+item_heat['collect']+item_heat['add-to-cart']+item_heat['payment']
item_heat.head(10)

,item_category,click,collect,add-to-cart,payment,heat
0,2,0.000100,-18.420681,-27.631021,-36.841361,-82.892963
1,3,0.693197,-18.420681,-27.631021,-36.841361,-82.199866
2,4,0.000100,-18.420681,-27.631021,-36.841361,-82.892963
3,6,2.197236,-18.420681,-27.631021,-36.841361,-80.695828
4,8,6.725034,4.394471,4.158958,5.545277,20.823741
5,10,5.017280,2.197291,6.591707,5.545277,19.351556
6,11,8.632484,9.043579,13.070130,6.437832,37.184026
7,12,4.969814,4.158908,7.917193,5.545277,22.591193
8,13,4.110876,3.583552,2.079592,7.167105,16.941124
9,15,6.079933,7.221841,4.158958,5.545277,23.006010


In [74]:
item_heat.sort_values(by='heat',ascending=False)[:5]

,item_category,click,collect,add-to-cart,payment,heat
1135,1863,12.399965,18.447303,27.370096,30.273517,88.490881
3707,5894,12.218179,17.926321,26.334053,27.387773,83.866326
4093,6513,12.113140,17.606247,26.352487,27.729792,83.801666
3393,5399,12.090140,17.579625,25.750066,27.749257,83.169088
8367,13230,12.290593,17.754764,26.045071,26.895330,82.985758


热度最高的前5商品类别的编号为：1863，5894，6513，5399，13230。

## 六. 总结及建议

1. PV、UV通常在21点左右达到峰值，因此如果在这个时候上新或者进行新活动，是最容易被用户关注到。
2. 上午11点左右通过平台、短信、邮件进行提醒购买，可以达到更好的效果。
3. 这年的双十二无论从访问量，活跃度还是转换率都达到了不错的效果。
4. 在用户首次购买间隔28天的时候可以对用户采取一定的营销措施。
5. 用户可分为普通活跃和非常活跃两类用户。
6. 商品编号为1863的热度最高。